# Criptosistema de McEliece
Breve tutorial sobre cómo implementar el criptosistema de McEliece en SageMath.

#### En primer lugar veamos cómo funciona el criptosistema para ver qué necesitamos definir:

La versión original del criptosistema McEliece dada por Robert J. McEliece, basado en códigos binarios de Goppa en el año 1978, se describe de la siguiente manera. Los valores de $\textbf{n}$, $\textbf{k}$ y $\textbf{t}$ son parámetros disponibles públicamente, pero $L$, $g$, $P$ y $S$ son secretos generados al azar. Entonces, este criptosistema de clave pública funciona con los siguientes pasos:

Paso 1: En primer lugar, Alice genera un par de claves públicas y privadas en función de los valores disponibles públicamente. Durante esto:

1. Alice selecciona un $\left[\textbf{n},\textbf{k}\right]-$código binario de Goppa, con su matriz generatriz G de dimensión $\textbf{k}\times\textbf{n}$, capaz de corregir $\textbf{t}$ errores.

2. Después, selecciona una matriz $S$ binaria, no singular, de dimensión $\textbf{k}\times\textbf{k}$ y una matriz permutación $P$ de dimensión $\textbf{n}\times\textbf{n}$.

3. Hace el cálculo de la matriz $\textbf{k}\times\textbf{n}$, $\hat{\textbf{G}}=S\cdot G\cdot P$.

4. Publica su clave pública: $\left(\hat{\textbf{G}},\textbf{t}\right)$.

5. Mantiene privada la clave: $\left(S,G,P\right)$.

Paso 2: Supongamos que Bob tiene que enviar un mensaje cifrado a Alice:

1. Bob tiene un mensaje binario de texto plano $m$ de longitud $k$.

2. Carga la clave pública de Alice: $\left(\hat{\textbf{G}},\textbf{t}\right)$.

3. Genera un vector aleatorio $z$ de $n-$bits con peso $w(z)=t$.

4. Bob calcula el texto cifrado $c=m\cdot\hat{\textbf{G}}+z$ y lo envía a Alice.

Paso 3: Supongamos que Alice ha recibido el texto cifrado $c$. Ella descifra el texto cifrado recibido como:

1. Alice calcula $P^{-1}$ usando su clave privada.

2. Multiplicamos el texto recibido por $P^{-1}$:$$c\cdot P^{-1}=m\cdot S\cdot G+z\cdot P^{-1}$$

3. Finalmente, utiliza el algoritmo de descodificación de los códigos Goppa para determinar el valor de $m$.

## Paso 1. Generamos un par de claves públicas y privadas.

#### 1. Necesitamos definir un código de Goppa.

Para definir un código de Goppa recurrimos a una de las librerías predefinidas en SageMath.

Definamos los parámetros iniciales para obtener un código de Goppa, donde $n$ es la dimensión, $\mathbb{F}_{q^m}$ el cuerpo donde están definidos los caracteres del mensaje y $t$ se corresponde al grado del polinomio generador del código, al ser un código de Goppa, este parámetro denota la capacidad de corrección del código.

In [1]:
n = 14
q = 2
m = 4
t = 2

F.<a>=GF(q**m);
PolRing=PolynomialRing(F,'x');
x=PolRing.gen();

Una vez que tenemos definidos los parámetros iniciales, el cuerpo en el que vamos a trabajar y la variable del polinomio, tenemos que construir un polinomio $g(x)$ de grado $t$ que sea irreducible. Para ello realizamos el siguiente proceso:

In [38]:
g = 0
check = True
while check:
    h = PolRing.random_element()
    if h.is_irreducible() and h.degree() == t and h[t]==1:
        g += h
        check = False
    else:
        None
g

x^2 + (a^3 + a^2 + a)*x + a

Al ser $g$ irreducible, para construir el conjunto $L$ nos basta con tomar una muestra aleatoria de tamaño $n$ de elementos del cuerpo $\mathbb{F}_{q^m}$. Otras definiciones de los códigos de Goppa no exigen que $g$ sea irreducible, con lo que añadimos que para construir $L$ hay que tomar los $\alpha_i \in \mathbb{F}_{q^m}$ tales que $g(\alpha_i)\neq 0$ $\forall i\in\{1,\ldots ,n\}$. Pasamos pues a construir $L$:

In [39]:
L = sample(F.list(),n); L

[a^3 + a^2,
 a^3 + a^2 + 1,
 a^3 + a,
 a^2 + a + 1,
 1,
 a^3,
 a^3 + a^2 + a,
 a^2 + 1,
 a + 1,
 a^2,
 a^3 + 1,
 a,
 0,
 a^3 + a^2 + a + 1]

Al tener $g(x)$ y $L$, que son parámetros necesarios, ya podemos construir un código de Goppa. Para ello, vamos a trabajar con métodos predefinidos de SageMath:

In [40]:
C = codes.GoppaCode(g,L); C

[14, 6] Goppa code over GF(2)

Como podemos ver, ya tenemos un $[n,k]-$código de Goppa $\mathcal{C}$ definido sobre $\mathbb{F}_{2}$.

In [41]:
G = C.generator_matrix(); G

[1 0 0 0 0 1 0 1 0 1 1 1 1 1]
[0 1 0 0 0 1 0 1 1 1 0 1 0 0]
[0 0 1 0 0 1 0 0 1 0 0 1 1 0]
[0 0 0 1 0 1 0 0 1 1 0 0 0 1]
[0 0 0 0 1 0 0 0 1 1 1 1 0 1]
[0 0 0 0 0 0 1 1 0 1 1 1 0 0]

Con el método C.generator_matrix() obtenemos una matriz generatriz del código.

In [42]:
H = C.parity_check_matrix(); H

[0 1 0 1 0 0 1 0 1 0 1 0 1 0]
[1 1 1 1 0 1 0 1 1 1 1 1 0 0]
[0 1 0 1 1 1 0 1 1 0 1 0 0 1]
[0 0 1 1 0 0 1 1 0 1 1 0 1 0]
[1 1 0 1 0 1 1 1 1 0 0 0 0 1]
[1 0 0 1 0 0 1 1 0 1 1 0 0 0]
[0 1 0 0 1 1 1 0 0 1 1 1 0 0]
[1 0 1 1 0 0 0 0 1 1 1 0 0 1]

Con el método C.parity_check_matrix() obtenemos una matriz de control del código.

Para obtener la clave pública, necesitamos calcular $k$ que es la dimensión del código:

In [43]:
k = C.dimension(); k

6

#### 2. Calculamos las clave pública y los parámetros privados:

Empezamos calculando una matriz aleatoria $S$ invertible:

In [44]:
S=random_matrix(GF(q),k);
while rank(S) < k:
    S = random_matrix(GF(q),k);
S

[0 0 1 0 0 1]
[0 1 0 1 0 1]
[1 0 1 0 1 1]
[0 0 0 0 1 1]
[0 0 1 1 0 1]
[1 1 1 0 1 1]

Y ahora calculamos una matriz de permutación $P$:

In [45]:
columnas = [0..(n-1)]
P = matrix(GF(q),n)
for i in range(n):
    posicion=randint(0,len(columnas)-1)
    P[i,columnas.pop(posicion)]=1
P

[0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Ya estamos en condiciones de poder calcular una clave pública de nuestro criptosistema:

In [46]:
Gp = S*G*P; Gp

[0 1 1 0 0 1 1 0 0 1 1 1 1 0]
[1 0 1 1 0 0 0 0 0 1 0 0 1 1]
[0 1 1 0 0 0 0 1 1 1 0 0 1 0]
[1 0 1 0 0 0 1 1 0 0 1 0 0 0]
[1 1 1 1 0 0 0 0 0 1 1 1 0 0]
[0 1 1 0 1 1 1 1 1 1 1 0 0 1]

La clave pública es la matriz Gp y t.

In [47]:
(Gp,t)

(
[0 1 1 0 0 1 1 0 0 1 1 1 1 0]   
[1 0 1 1 0 0 0 0 0 1 0 0 1 1]   
[0 1 1 0 0 0 0 1 1 1 0 0 1 0]   
[1 0 1 0 0 0 1 1 0 0 1 0 0 0]   
[1 1 1 1 0 0 0 0 0 1 1 1 0 0]   
[0 1 1 0 1 1 1 1 1 1 1 0 0 1], 2
)

Alice (o nosotros) tenemos el par de claves públicas $\left(\hat{\textbf{G}},\textbf{t}\right)$ y mantenemos privadas $(G,S,P)$.

## Paso 2: Supongamos que Bob tiene que enviar un mensaje cifrado a Alice:

1. Bob tiene un mensaje binario de texto plano $m$ de longitud $k$. (En este caso lo vamos a generar de forma aleatoria como ejemplo).

In [48]:
mensaje = vector([GF(q).random_element() for ii in range(C.dimension())]); mensaje

(0, 0, 0, 1, 1, 1)

2. Carga la clave pública de Alice: $\left(\hat{\textbf{G}},\textbf{t}\right)$.

In [49]:
(Gp,t)

(
[0 1 1 0 0 1 1 0 0 1 1 1 1 0]   
[1 0 1 1 0 0 0 0 0 1 0 0 1 1]   
[0 1 1 0 0 0 0 1 1 1 0 0 1 0]   
[1 0 1 0 0 0 1 1 0 0 1 0 0 0]   
[1 1 1 1 0 0 0 0 0 1 1 1 0 0]   
[0 1 1 0 1 1 1 1 1 1 1 0 0 1], 2
)

3. Genera un vector aleatorio $z$ de $n-$bits con peso $w(z)=t$.

Una de las varias fortalezas de este criptosistema es que codificamos el mensaje añadiendo un error con peso menor o igual que $t$.

In [50]:
error = [GF(q)(0) for ii in range(n)]
for ii in range(t): # Con esto aseguramos que el error tenga peso <= t
    error[ZZ.random_element(n)] = GF(q).random_element()

z = vector(error); z

(0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)

4. Bob calcula el texto cifrado $c=m\cdot\hat{\textbf{G}}+z$ y lo envía a Alice.

In [51]:
c = mensaje*Gp+z; c

(0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1)

## Paso 3: Supongamos que Alice ha recibido el texto cifrado $c$. Ella descifra el texto cifrado recibido como:

1. Alice calcula $P^{-1}$ usando su clave privada.

In [52]:
Pinv = P.inverse(); Pinv

[0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0]

2. Multiplicamos el texto recibido por $P^{-1}$:$$c\cdot P^{-1}=m\cdot S\cdot G+z\cdot P^{-1}$$

In [53]:
cnew = c*Pinv; cnew

(1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0)

3. Finalmente, utiliza el algoritmo de descodificación de los códigos Goppa para determinar el valor de $m$.

Paso 1: Descodificamos el mensaje cnew. Para ello recurrimos al método predefinido C.decode_to_code().

In [54]:
x1 = C.decode_to_code(cnew); x1

(1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0)

Paso 2: Nos quedamos con las primeras $k$ componentes de $x_1$, las restantes $n-k$ componentes son añadidas simplemente para que el mensaje codificado tenga longitud $n$ no aportando nada de información.

In [55]:
x0 = vector([x1[i] for i in range(0,C.dimension())]); x0

(1, 1, 0, 1, 0, 1)

Paso 3: Recuperamos el mensaje original sin errores.

In [56]:
x0*S.inverse()

(0, 0, 0, 1, 1, 1)

Y como se puede observar, hemos desencriptado (y corregido) nuestro mensaje encriptado.

#### Comentarios.